# Import

In [30]:
import pandas as pd
from db.mongo import MyMongo

# Load Data

In [176]:
with MyMongo() as db:
    external = db.get_df_from_table('public_website', 'a6_ax_exe_external')
    in_page = db.get_df_from_table('public_website', 'b1_a1_ax_exe_in_page')
    website = db.get_df_from_table('public_website', 'website_all')

<--Mongo Connected.
Mongo Connection Closed.-->


# Index and Columns

In [60]:
# idx_ext_ax = external['ax_code'].apply(lambda x: bool(len(x)))
idx_ext_ax = list(map(lambda x: bool(len(x)), external['ax_code']))
idx_ext_exe = list(map(lambda x: bool(len(x)), external['exe_code']))
idx_in_ax = list(map(lambda x: bool(len(x)), in_page['ax_code']))
idx_in_exe = list(map(lambda x: bool(len(x)), in_page['exe_code']))

In [145]:
cols_code = ['url', 'ax_code', 'exe_code']
cols_file = ['url', 'ax_file', 'exe_file']

# Functions

In [68]:
def split_list_and_get_file_name(li):
    return [l.split('/')[-1] for l in li]

# Transform

## Split List and Get File Name

In [70]:
external['ax_file'] = external['ax_code'].apply(split_list_and_get_file_name)
external['exe_file'] = external['exe_code'].apply(split_list_and_get_file_name)
in_page['ax_file'] = in_page['ax_code'].apply(split_list_and_get_file_name)
in_page['exe_file'] = in_page['exe_code'].apply(split_list_and_get_file_name)

## Expand list to df

In [97]:
# from itertools import zip_longest
# a = 1
# b = [2,3]
# list(zip_longest([a], b, fillvalue=a))
# input df output df
def make_expanded_df_from_list_df(list_df, col_name):
    from itertools import zip_longest
    result = []
    for i, row in list_df.iterrows():
        url = row['url']
        files = row[col_name]
        if files:
            tmp = list(zip_longest([url], files, fillvalue=url))
            result.extend(tmp)
    return pd.DataFrame(columns=['url', col_name], data=result)

In [112]:
df_ext_ax_expanded = make_expanded_df_from_list_df(external.loc[idx_ext_ax, ['url', 'ax_file']], 'ax_file')
df_ext_exe_expanded = make_expanded_df_from_list_df(external.loc[idx_ext_exe, ['url', 'exe_file']], 'exe_file')
df_in_ax_expanded = make_expanded_df_from_list_df(in_page.loc[idx_in_ax, ['url', 'ax_file']], 'ax_file')
df_in_exe_expanded = make_expanded_df_from_list_df(in_page.loc[idx_in_exe, ['url', 'exe_file']], 'exe_file')

## Add domain

In [128]:
from urllib.parse import urlsplit
# df_ext_ax_expanded.loc[::,['domain', 'path']] = df_ext_ax_expanded['url'].apply(lambda x: urlsplit(x)[1:3])
df_ext_ax_expanded['domain'] = df_ext_ax_expanded['url'].apply(lambda x: urlsplit(x)[1])
df_ext_ax_expanded['path'] = df_ext_ax_expanded['url'].apply(lambda x: urlsplit(x)[2])

df_ext_exe_expanded['domain'] = df_ext_exe_expanded['url'].apply(lambda x: urlsplit(x)[1])
df_ext_exe_expanded['path'] = df_ext_exe_expanded['url'].apply(lambda x: urlsplit(x)[2])

df_in_ax_expanded['domain'] = df_in_ax_expanded['url'].apply(lambda x: urlsplit(x)[1])
df_in_ax_expanded['path'] = df_in_ax_expanded['url'].apply(lambda x: urlsplit(x)[2])

df_in_exe_expanded['domain'] = df_in_exe_expanded['url'].apply(lambda x: urlsplit(x)[1])
df_in_exe_expanded['path'] = df_in_exe_expanded['url'].apply(lambda x: urlsplit(x)[2])
# df_ext_ax_expanded['domain'] = df_ext_ax_expanded['url']

## Add Name & Description

In [202]:
def add_name_and_description(df):
    for i, row in df.iterrows():
        domain = row['domain']
        found = website.loc[website['url'].str.contains(domain)]
        if len(found):
            df.at[i, 'name'] = found.iloc[0]['name']
            df.at[i, 'description'] = found.iloc[0]['description']

In [204]:
add_name_and_description(df_ext_exe_expanded)
add_name_and_description(df_ext_ax_expanded)
add_name_and_description(df_in_exe_expanded)
add_name_and_description(df_in_ax_expanded)

### Index and Columns

In [209]:
cols_domain_ax = ['domain', 'name', 'path', 'ax_file']
cols_domain_exe = ['domain', 'name', 'path', 'exe_file']

In [162]:
idx_ext_exe_gov = df_ext_exe_expanded['domain']=='www.gov.kr'
idx_ext_ax_gov = df_ext_ax_expanded['domain']=='www.gov.kr'

## Groupby domain

In [210]:
grp_ext_exe = df_ext_exe_expanded.groupby('domain').agg('count').sort_values('exe_file', ascending=False)
grp_in_exe = df_in_exe_expanded.groupby('domain').agg('count').sort_values('exe_file', ascending=False)

grp_ext_ax = df_ext_ax_expanded.groupby('domain').agg('count').sort_values('ax_file', ascending=False)
grp_in_ax = df_in_ax_expanded.groupby('domain').agg('count').sort_values('ax_file', ascending=False)

# Border ---------------------------------

In [205]:
df_ext_exe_expanded.head()

,url,exe_file,domain,path,name,description
0,http://www.kops.or.kr/AnySign/anySign4PCInterf...,hConvert2pfx.exe,www.kops.or.kr,/AnySign/anySign4PCInterface.js,환자안전 보고학습시스템 포털,환자안전 보고학습시스템 업무포털시스템
1,http://www.kops.or.kr/AnySign/anySign4PCInterf...,xwcup_install_windows_x64.exe,www.kops.or.kr,/AnySign/anySign4PCInterface.js,환자안전 보고학습시스템 포털,환자안전 보고학습시스템 업무포털시스템
2,http://www.kops.or.kr/AnySign/anySign4PCInterf...,xwcup_install_windows_x86.exe,www.kops.or.kr,/AnySign/anySign4PCInterface.js,환자안전 보고학습시스템 포털,환자안전 보고학습시스템 업무포털시스템
3,http://gangwon.childcare.go.kr/html/Xecure/xec...,xwcup_install_windows_x64.exe,gangwon.childcare.go.kr,/html/Xecure/xecureweb_up.js,강원도육아종합지원센터,"강원도육아종합지원센터의 홈페이지로 센터 소개, 영유아가정지원, 어린이집지원, 정보마..."
4,http://gangwon.childcare.go.kr/html/Xecure/xec...,xwcup_install_windows_x86.exe,gangwon.childcare.go.kr,/html/Xecure/xecureweb_up.js,강원도육아종합지원센터,"강원도육아종합지원센터의 홈페이지로 센터 소개, 영유아가정지원, 어린이집지원, 정보마..."


In [206]:
df_ext_exe_expanded.loc[idx_ext_exe_gov]

,url,exe_file,domain,path,name,description
120,http://www.gov.kr/webPlugins/nlogin/AnySign4PC...,xwcup_install_windows_x64.exe,www.gov.kr,/webPlugins/nlogin/AnySign4PC/anySign4PCInterf...,정부24,"정부24 홈페이지로 정부서비스, 민원서비스, 정책정보 등의 서비스를 제공합니다."
121,http://www.gov.kr/webPlugins/nlogin/AnySign4PC...,xwcup_install_windows_x86.exe,www.gov.kr,/webPlugins/nlogin/AnySign4PC/anySign4PCInterf...,정부24,"정부24 홈페이지로 정부서비스, 민원서비스, 정책정보 등의 서비스를 제공합니다."
122,http://www.gov.kr/wizvera/veraport/veraport20.js,veraport-g3-x64.exe,www.gov.kr,/wizvera/veraport/veraport20.js,정부24,"정부24 홈페이지로 정부서비스, 민원서비스, 정책정보 등의 서비스를 제공합니다."
123,http://www.gov.kr/wizvera/veraport/veraport20.js,veraport-g3-x64-sha2.exe,www.gov.kr,/wizvera/veraport/veraport20.js,정부24,"정부24 홈페이지로 정부서비스, 민원서비스, 정책정보 등의 서비스를 제공합니다."
124,http://www.gov.kr/wizvera/veraport/veraport20.js,veraport-g3.exe,www.gov.kr,/wizvera/veraport/veraport20.js,정부24,"정부24 홈페이지로 정부서비스, 민원서비스, 정책정보 등의 서비스를 제공합니다."
125,http://www.gov.kr/wizvera/veraport/veraport20.js,veraport-g3-sha2.exe,www.gov.kr,/wizvera/veraport/veraport20.js,정부24,"정부24 홈페이지로 정부서비스, 민원서비스, 정책정보 등의 서비스를 제공합니다."


In [164]:
df_ext_ax_expanded.loc[idx_ext_ax_gov, cols_domain_ax]

,domain,path,ax_file
75,www.gov.kr,/webPlugins/nlogin/AnySign4PC/anySign4PCInterf...,xwcup_install_windows_x86.cab
76,www.gov.kr,/webPlugins/nlogin/AnySign4PC/anySign4PCInterf...,xwcup_install_windows_x64.cab
77,www.gov.kr,/wizvera/veraport/veraport20.js,veraport20-x64-sha2.cab
78,www.gov.kr,/wizvera/veraport/veraport20.js,veraport20-x64.cab
79,www.gov.kr,/wizvera/veraport/veraport20.js,veraport20-sha2.cab
80,www.gov.kr,/wizvera/veraport/veraport20.js,veraport20.cab


In [175]:
# grp_ext_exe.loc[grp_ext_exe['url']==1].index
df_ext_exe_expanded.merge(grp_ext_exe.loc[grp_ext_exe['url']==1, ['path']], left_on='domain', right_index=True, how='inner')

,url,exe_file,domain,path_x,path_y
5,http://museum.navy.ac.kr/common/js/embedJs.js,wmpfirefoxplugin.exe,museum.navy.ac.kr,/common/js/embedJs.js,1
6,http://klis.jeonnam.go.kr/sis/gpkisecureweb/va...,install_off_v1.0.4.9.exe,klis.jeonnam.go.kr,/sis/gpkisecureweb/var.js,1
7,http://www.cne.go.kr/kcase/EPKICommon.js,KCaseAgent_Installer_v1.3.13.exe,www.cne.go.kr,/kcase/EPKICommon.js,1
8,http://www.cntae.go.kr/include/EPKI/kcase/EPKI...,KCaseAgent_Installer.exe,www.cntae.go.kr,/include/EPKI/kcase/EPKICommon.js,1
29,http://www.cnssed.go.kr/include/EPKI/kcase/EPK...,KCaseAgent_Installer.exe,www.cnssed.go.kr,/include/EPKI/kcase/EPKICommon.js,1
30,http://gdoc.go.kr/gpki/gpkisecureweb/var.js,install_off_v1.0.4.9.exe,gdoc.go.kr,/gpki/gpkisecureweb/var.js,1
34,http://jangyubranch.gimhae.go.kr/share/js/all.js,jre-1_5_0_11-windows-i586-p-s.exe,jangyubranch.gimhae.go.kr,/share/js/all.js,1
35,http://hnrl.namgu.incheon.kr/js/usr/common.js,wmpfirefoxplugin.exe,hnrl.namgu.incheon.kr,/js/usr/common.js,1
42,http://www.bppl.or.kr/js/usr/common.js,wmpfirefoxplugin.exe,www.bppl.or.kr,/js/usr/common.js,1
43,http://tour.gimje.go.kr/wenUploader/wenplayer.js,flashplayer11_9r900_170_winax.exe,tour.gimje.go.kr,/wenUploader/wenplayer.js,1


In [138]:
grp_ext_exe.merge(grp_in_exe, right_index=True, left_index=True)

,url_x,exe_file_x,path_x,url_y,exe_file_y,path_y
domain,,,,,,
www.hs-support.go.kr,1,1,1,1,1,1


In [140]:
grp_ext_ax.merge(grp_in_ax, right_index=True, left_index=True)

,url_x,ax_file_x,path_x,url_y,ax_file_y,path_y
domain,,,,,,
minwon.nps.or.kr,2,2,2,2,2,2


In [211]:
grp_ext_exe

,url,exe_file,path,name,description
domain,,,,,
www.gov.kr,6,6,6,6,6
exim.allbaro.or.kr,4,4,4,4,4
total.kcomwel.or.kr,3,3,3,3,3
www.efine.go.kr,3,3,3,3,3
apply.lh.or.kr,3,3,3,3,3
ebid.kr.or.kr,3,3,3,3,3
ebid.knoc.co.kr,3,3,3,3,3
water-pos.kwater.or.kr,3,3,3,3,3
www.kops.or.kr,3,3,3,3,3


In [141]:
grp_in_exe

,url,exe_file,path
domain,,,
mail.edunavi.kr:443,2,2,2
www.ktcu.or.kr,2,2,2
www.namis.or.kr,2,2,2
www.ongjin.go.kr,2,2,2
cals.icpa.or.kr,1,1,1
edu.gg.go.kr,1,1,1
etax.busan.go.kr,1,1,1
sso.jnei.go.kr,1,1,1
www.e-hipassplus.co.kr,1,1,1


In [142]:
grp_ext_ax

,url,ax_file,path
domain,,,
www.gov.kr,6,6,6
www.gyo6.net,4,4,4
hanoischool.net,3,3,3
buyeosaeil.kr,3,3,3
www.hanoischool.net,3,3,3
www.mpki.go.kr,3,3,3
land.kwater.or.kr,2,2,2
gyeongnam.childcare.go.kr,2,2,2
www.kops.or.kr,2,2,2


In [143]:
grp_in_ax

,url,ax_file,path
domain,,,
g-money.gg.go.kr,2,2,2
minwon.nps.or.kr,2,2,2
ebid.knoc.co.kr,1,1,1
www.gangjinsports.com,1,1,1
www.gochang.go.kr,1,1,1
www.jntv.go.kr,1,1,1
www.ksmc.or.kr,1,1,1


In [100]:
df_ext_exe_expanded.groupby('exe_file').agg('count').sort_values('url', ascending=False)

,url
exe_file,
KCaseAgent_Installer.exe,17
xwcup_install_windows_x86.exe,16
xwcup_install_windows_x64.exe,16
GPKISecureWebX.exe,14
TouchEn_nxKey_Installer_32bit.exe,12
TouchEn_nxKey_Installer_64bit.exe,12
GPKISecureWebXPlugin.exe,8
GPKISecureWebXPlugin64.exe,8
wmpfirefoxplugin.exe,7


In [104]:
df_in_exe_expanded.groupby('exe_file').agg('count').sort_values('url', ascending=False)

,url
exe_file,
astx_setup.exe,4
GPKISecureWebX.exe,2
CX60_Plugin_u_setup.exe,1
CX60u_OCX_setup.exe,1
HTML5v1.3.7.exe,1
ISignPlusWA_Setup_3.0.13.1.exe,1
JXCealNT.exe,1
JXCealPlugin.exe,1
MiPlatform_SetupDeploy320U_200907.exe,1


In [77]:
external.loc[idx_ext_ax, cols_file]

,url,ax_file,exe_file
0,http://www.kops.or.kr/AnySign/anySign4PCInterf...,"[xwcup_install_windows_x86.cab, xwcup_install_...","[hConvert2pfx.exe, xwcup_install_windows_x64.e..."
17,http://gangwon.childcare.go.kr/html/Xecure/xec...,"[xwcup_install_windows_x64.cab, xwcup_install_...","[xwcup_install_windows_x64.exe, xwcup_install_..."
45,http://sll.seoul.go.kr/js/framework/ui/clipboa...,[swflash.cab],[]
70,http://council.chilgok.go.kr/public/java/desig...,[swflash.cab],[]
72,http://library.chilgok.go.kr/./js/newCommon.js,[swflash.cab],[]
80,http://ydespecial.kr/./js/newCommon.js,[swflash.cab],[]
86,http://lib.cd.go.kr/./js/newCommon.js,[swflash.cab],[]
97,http://lib.jp.go.kr/./jslib/path_fun.js,[swflash.cab],[]
104,http://wdoc.snuh.org/login/cqweb.inc,[TrustWeb.cab],[]
113,http://traffic.gccity.go.kr/js/common/common.js,[swflash.cab],[]


In [67]:
external.loc[idx_ext_exe, 'exe_code'].apply(split_list_and_get_file_name)

0       [hConvert2pfx.exe, xwcup_install_windows_x64.e...
17      [xwcup_install_windows_x64.exe, xwcup_install_...
28                                 [wmpfirefoxplugin.exe]
31                             [install_off_v1.0.4.9.exe]
35                     [KCaseAgent_Installer_v1.3.13.exe]
36                             [KCaseAgent_Installer.exe]
40                     [.test(pinfo.exe, .test(pinfo.exe]
55      [TouchEn_nxKey_Installer_64bit.exe, TouchEn_nx...
62      [TouchEn_nxKey_Installer_64bit.exe, TouchEn_nx...
107     [GPKISecureWebXPlugin.exe, GPKISecureWebXPlugi...
112     [GPKISecureWebXPlugin.exe, GPKISecureWebXPlugi...
118     [GPKISecureWebXPlugin.exe, GPKISecureWebXPlugi...
122     [GPKISecureWebXPlugin.exe, GPKISecureWebXPlugi...
127     [TouchEn_nxKey_Installer_64bit.exe, TouchEn_nx...
128     [TouchEn_nxKey_Installer_64bit.exe, TouchEn_nx...
129     [TouchEn_nxKey_Installer_64bit.exe, TouchEn_nx...
135                            [KCaseAgent_Installer.exe]
139           

In [61]:
external.loc[idx_ext_ax, cols]

,url,ax_code,exe_code
0,http://www.kops.or.kr/AnySign/anySign4PCInterf...,"[install/xwcup_install_windows_x86.cab, instal...","[download/hConvert2pfx.exe, install/xwcup_inst..."
17,http://gangwon.childcare.go.kr/html/Xecure/xec...,[activex/Xecure/xw/xwcup_install_windows_x64.c...,[activex/Xecure/xw/xwcup_install_windows_x64.e...
45,http://sll.seoul.go.kr/js/framework/ui/clipboa...,[pub/shockwave/cabs/flash/swflash.cab],[]
70,http://council.chilgok.go.kr/public/java/desig...,[flash/swflash.cab],[]
72,http://library.chilgok.go.kr/./js/newCommon.js,[flash/swflash.cab],[]
80,http://ydespecial.kr/./js/newCommon.js,[flash/swflash.cab],[]
86,http://lib.cd.go.kr/./js/newCommon.js,[flash/swflash.cab],[]
97,http://lib.jp.go.kr/./jslib/path_fun.js,[flash/swflash.cab],[]
104,http://wdoc.snuh.org/login/cqweb.inc,[CQWeb/TrustWeb.cab],[]
113,http://traffic.gccity.go.kr/js/common/common.js,[/fpdownload.macromedia.com/pub/shockwave/cabs...,[]


In [7]:
in_page.loc[::, cols].head()

,url,ax_code,exe_code
0,http://www.butterflyhp.or.kr/main/,[],[]
1,http://butterflyhp.or.kr/application/member/lo...,[],[]
2,http://www.butterflyhp.or.kr/application/membe...,[],[]
3,http://own.kribb.re.kr/main/main.jsp,[],[]
4,http://edu.kribb.re.kr/main/main.jsp,[],[]


In [63]:
external.loc[::, cols]. head()

,url,ax_code,exe_code
0,http://www.kops.or.kr/AnySign/anySign4PCInterf...,"[install/xwcup_install_windows_x86.cab, instal...","[download/hConvert2pfx.exe, install/xwcup_inst..."
1,http://www.gwangju.go.kr/home/js/global.js,[],[]
2,http://michu.incheon.kr/share/js/base.js,[],[]
3,http://michu.incheon.kr/share/js/base_subong.js,[],[]
4,http://michu.incheon.kr/ShareEtc/chk.js,[],[]
